# 2022-10-17 • General simulator software design

In the previous notebook, the firing rate error in the N-to-1 simulations was fixed. We want to know re-run those simulations with actual lognormal Poisson inputs.

When writing the network simulation code, the N-to-1 simulation code was copied and adapted.
I.e. there is duplication in functionality, and divergence in their APIs.
It's time for consolidation.
Advantage: easier to also investigate LIF/EIF neurons, different neuron types, etc.

## Imports

In [110]:
#

In [111]:
using MyToolbox

In [112]:
using VoltoMapSim

## .

In [115]:
izh = @eqs begin
    
    dv/dt = (k*(v-vᵣ)*(v-vₜ) - u - I_syn + I_ext) / C
    du/dt = a*(b*(v-vᵣ) - u)

    I_syn = gₑ*(v-Eₑ) + gᵢ*(v-Eᵢ)

    dgₑ/dt = -gₑ / τ  # Represents sum over all exc synapses
    dgᵢ/dt = -gᵢ / τ
    
    @spike if v > v_peak
        v = v_reset
        u += Δu
    end
end;

Test Failed at C:\Users\tfiers\phd\pkg\SpikeLab\src\eqparse.jl:70
  Expression: eq.head == :(=)
   Evaluated: macrocall == var"="


LoadError: LoadError: ArgumentError: Could not parse the given model.
in expression starting at In[115]:1

In [26]:
izh

SpikeLab.Model2
 with variables {v, u, I_syn, gₑ, gᵢ}
 and parameters {C, Eᵢ, Eₑ, I_ext, a, b, k, vᵣ, vₜ, τ}


In [27]:
izh.generated_func

:((diff, vars, params)->begin
          @unpack (v, u, I_syn, gₑ, gᵢ) = vars
          @unpack (C, Eᵢ, Eₑ, I_ext, a, b, k, vᵣ, vₜ, τ) = params
          diff.v = (((k * (v - vᵣ) * (v - vₜ) - u) - I_syn) + I_ext) / C
          diff.u = a * (b * (v - vᵣ) - u)
          vars.I_syn = gₑ * (v - Eₑ) + gᵢ * (v - Eᵢ)
          diff.gₑ = -gₑ / τ
          diff.gᵢ = -gᵢ / τ
      end)

In [29]:
vars = CVec{Float64}(v=0, u=0, I_syn=0, gₑ=0, gᵢ=0)
diff = similar(vars)
params = idvec(:C, :Eᵢ, :Eₑ, :I_ext, :a, :b, :k, :vᵣ, :vₜ, :τ)
params = similar(params, Float64)
params .= 1
izh.f(diff, vars, params)
diff

ComponentVector{Float64}(v = 2.0, u = -1.0, I_syn = 1.3145426e-315, gₑ = -0.0, gᵢ = -0.0)

In [38]:
vars .+= diff * 0.1 #ms

ComponentVector{Float64}(v = 0.2, u = -0.1, I_syn = 1.3145426e-316, gₑ = 0.0, gᵢ = 0.0)

In [39]:
vars

ComponentVector{Float64}(v = 0.2, u = -0.1, I_syn = 1.3145426e-316, gₑ = 0.0, gᵢ = 0.0)

In [40]:
izh.f()

LoadError: MethodError: no method matching (::var"#7#8")()
[0mClosest candidates are:
[0m  (::var"#7#8")([91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at C:\Users\tfiers\phd\pkg\SpikeLab\src\eqparse.jl:61

In [ ]:
params_any = similar(params, Any)
params_any .= params
params_any.C = "wrongtype"
izh.f(diff, vars, params_any)

In [32]:
using Unitful: mV, nS

In [34]:
3mV

3 mV